In [ ]:
from plotly import express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
import cv2

In [ ]:
from epi.plotlyx.express import scatter_3d
from epi.plotlyx.utils.fig import np_to_plotly
from epi.plotlyx.render import CameraCoordinateRenderer, render_camera_axes

In [ ]:
from epi import geometry as geom

In [ ]:
from typing import Union, List, Tuple

In [ ]:
import numpy as np
from IPython.core.debugger import set_trace

In [ ]:
from epi.camera import ProjCamera
from epi.model import read_vertices, Model

In [ ]:
PLOTLY_UP = dict(x=0, y=1, z=0)
PLOTLY_CENTER = dict(x=0, y=0, z=0)
PLOTLY_EYE = dict(x=0, y=0, z=2)
SCENE_SCALE = 15

In [ ]:
proj_camera1 = ProjCamera(
    np.array([0, 0, 5]),
    0.2,
    640,
    480,
    0,
    0,
    0.001,
    0.001,
)

proj_camera2 = ProjCamera(
    np.array([1, 0, 5]),
    0.2,
    640,
    480,
    0,
    0,
    0.001,
    0.001,
)

In [ ]:
vertices = read_vertices("./building_04.obj")
house_model = Model(vertices, color=vertices[:, 1], position=np.array([0, 0, 10]))

In [ ]:
proj_camera1.look_at(house_model.center)
proj_camera2.look_at(house_model.center)

In [ ]:
fig = go.Figure(
    data=[
        scatter_3d(
            house_model.vertices,
            marker=dict(color=house_model.color, symbol="x", size=1),
        )
    ],
)

fig.update_layout(
    scene=dict(
        camera=dict(
            up=PLOTLY_UP,
        ),
        xaxis=dict(
            nticks=20,
            range=[-SCENE_SCALE, SCENE_SCALE],
        ),
        yaxis=dict(
            nticks=20,
            range=[-SCENE_SCALE, SCENE_SCALE],
        ),
        zaxis=dict(
            nticks=20,
            range=[-SCENE_SCALE, SCENE_SCALE],
        ),
    ),
    width=600,
    height=600,
    showlegend=False,
    margin=dict(r=20, l=10, b=10, t=10),
)


render_camera_axes(fig, proj_camera2)
render_camera_axes(fig, proj_camera1)
fig.update_scenes(aspectmode="data")

fig

In [ ]:
fig = go.Figure(
    layout=dict(
        scene=dict(
            camera=dict(
                up=PLOTLY_UP,
            ),
            xaxis=dict(
                nticks=20,
                range=[-SCENE_SCALE, SCENE_SCALE],
            ),
            yaxis=dict(
                nticks=20,
                range=[-SCENE_SCALE, SCENE_SCALE],
            ),
            zaxis=dict(
                nticks=20,
                range=[SCENE_SCALE, -SCENE_SCALE],
            ),
        ),
        width=600,
        height=600,
        showlegend=False,
        margin=dict(r=20, l=10, b=10, t=10),
    )
)

In [ ]:
camera1_renderer = CameraCoordinateRenderer(proj_camera1, fig=fig)
camera2_renderer = CameraCoordinateRenderer(proj_camera2, fig=fig)

In [ ]:
camera1_renderer.render(house_model)

In [ ]:
camera2_renderer.render(house_model)

In [ ]:
img1, depth1, world_idx1 = proj_camera1.render_img(house_model.vertices, color=1)
img2, depth2, world_idx2 = proj_camera2.render_img(house_model.vertices, color=1)

In [ ]:
from epi.image import normalize, ops as img_ops

In [ ]:
depth2_3c = img_ops.to_3channels(255 * normalize.min_max(depth2))
depth1_3c = img_ops.to_3channels(255 * normalize.min_max(depth1))

In [ ]:
fig = make_subplots(1, 2)
fig.add_trace(go.Image(z=depth1_3c), 1, 1)
fig.add_trace(go.Image(z=depth2_3c), 1, 2)
fig

In [ ]:
in_img1_idxs = set(world_idx1[np.where(~np.isnan(world_idx1))].astype(np.int32))
in_img2_idxs = set(world_idx2[np.where(~np.isnan(world_idx2))].astype(np.int32))

In [ ]:
epipolar_indices = np.array(list(in_img1_idxs.intersection(in_img2_idxs)))
epipolar_vertices = house_model.vertices[epipolar_indices]
epipolar_color = house_model.color[epipolar_indices]

In [ ]:
x_min, y_min, z_min = np.min(epipolar_vertices, axis=0)
x_max, y_max, z_max = np.max(epipolar_vertices, axis=0)

In [ ]:
go.Figure(
    data=[
        scatter_3d(
            epipolar_vertices,
            marker=dict(color=house_model.color[epipolar_indices], symbol="x", size=1),
        )
    ],
    layout=dict(
        scene=dict(
            camera=dict(
                up=PLOTLY_UP,
            ),
            xaxis=dict(
                nticks=20,
                range=[x_min, x_max],
            ),
            yaxis=dict(
                nticks=20,
                range=[y_min, y_max],
            ),
            zaxis=dict(
                nticks=20,
                range=[z_min, z_max],
            ),
        ),
        width=600,
        height=600,
        showlegend=False,
    ),
)

In [ ]:
st_kps = proj_camera1.project_vertices(epipolar_vertices, drop_last=True)
nd_kps = proj_camera2.project_vertices(epipolar_vertices, drop_last=True)

In [ ]:
F, mask = cv2.findFundamentalMat(np.int32(st_kps), np.int32(nd_kps))

In [ ]:
def compute_y_at_x(x, line):
    return (-line[2] - line[0] * x) / line[1]

In [ ]:
lines = (F @ geom.to_homogenous(nd_kps, axis=1).T).T

In [ ]:
st_lines = (F @ geom.to_homogenous(nd_kps, axis=1).T).T

In [ ]:
nd_lines = geom.to_homogenous(st_kps, axis=1) @ F

In [ ]:
def make_lines(lines: np.ndarray, points: np.ndarray, xlim: Tuple[int, int]):
    xmin, xmax = xlim
    go_lines = [
        go.Scatter(
            x=[xmin, xmax],
            y=[compute_y_at_x(xmin, line), compute_y_at_x(xmax, line)],
            mode="lines",
        )
        for line in lines
    ]
    return go_lines

In [ ]:
fig = go.Figure()
for ba in make_lines(st_lines[:100], lines, xlim=[0, 640]):
    fig.add_trace(ba)

fig.add_trace(
    go.Scatter(
        x=st_kps[:, 0], y=st_kps[:, 1], mode="markers", marker=dict(symbol="x", size=1)
    )
)

fig

In [ ]:
fig = go.Figure()
for ba in make_lines(nd_lines[:100], lines, xlim=[0, 640]):
    fig.add_trace(ba)

fig.add_trace(
    go.Scatter(
        x=nd_kps[:, 0], y=nd_kps[:, 1], mode="markers", marker=dict(symbol="x", size=1)
    )
)

fig

1. add axis to functions for homogenuous transformations  Done
2. Add visualization for points on the epipolar lines
3. Compute essential matrix
4. Use essential matrix to compute camera matrices for the two cameras
5. Project points back to 3d